In [2]:
%matplotlib inline
import pickle
import numpy as np
import pandas as pd
import math
import torch
from torch import nn
from sklearn import metrics

In [2]:
def imputeDataMean(scliedData, MaskMat, meanvalue):
    imputedata = []
    for i in range(len(scliedData)):
        a = np.array(scliedData[i])
        b = np.array(scliedData[i]) * np.array(MaskMat[i])
        e = b.sum(axis=0)
        c = np.array(MaskMat[0])
        d = c.sum(axis=0)
        d[d==0]=1
        mean = b.sum(axis=0)/d
        for j in range(a.shape[1]):
            if np.all(a[:,j] == -1):
                a[:,j]= meanvalue[j]
            else:
                k = a[:,j]
                k[k==-1]= e[j]
        imputedata.append(a)
    return imputedata

In [3]:
def imputeDataNearest(sliceData, meanvalue):
    imputedata = []
    for i in range(len(sliceData)):
        a = np.array(sliceData[i])
        b = pd.DataFrame(np.array(a))
        b = b.replace(-1,np.nan)
        c = b.count()
        for k in range(a.shape[1]):
            if c[k] == 0:
                b[k] = meanvalue[k]
            if c[k] == 1:
                b[k] = b[k].interpolate(method ='ffill',axis=0)
                b[k] = b[k].interpolate(method ='bfill',axis=0)
        b = b.interpolate(method ='nearest',axis=0,limit_direction ='both')
        b = b.interpolate(method ='ffill')
        b = b.interpolate(method ='bfill')
        b = b.values
        imputedata.append(b)
    return imputedata

In [4]:
def imputeLast(sliceData, meanvalue):
    imputedata = []
    for i in range(len(sliceData)):
        a = np.array(sliceData[i])
        for j in range(a.shape[1]):
            if np.all(a[:,j] == -1):
                a[:,j]= meanvalue[j]
        b = pd.DataFrame(np.array(a))
        b = b.replace(-1,np.nan)
        b = b.interpolate(method ='ffill',axis=0)
        b = b.interpolate(method ='bfill',axis=0)
        b = b.values
        imputedata.append(b)
    return imputedata

In [5]:
f = open('E:/WashU/Research/ICU/Data/train/X_train_sliced_norm.pkl','rb')
X_train_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/y_train.pkl','rb')
y_train = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_delta_mat.pkl','rb')
train_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_mask_mat.pkl','rb')
train_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/val/X_val_sliced_norm.pkl','rb')
X_val_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/y_val.pkl','rb')
y_val = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_delta_mat.pkl','rb')
val_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_mask_mat.pkl','rb')
val_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/test/X_test_sliced_norm.pkl','rb')
X_test_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/y_test.pkl','rb')
y_test = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/test_delta_mat.pkl','rb')
test_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/test/test_mask_mat.pkl','rb')
test_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/mean_norm.pkl','rb')
mean = pickle.load(f)
f.close()

In [64]:
X_train = np.array(imputeDataMean(X_train_sliced, train_mask_mat, mean))
X_val = np.array(imputeDataMean(X_val_sliced, val_mask_mat, mean))

In [65]:
y_train = np.array(y_train)
X_resampled = []
y_resampled = []
y_one_index = np.argwhere(y_train==1).reshape(-1)
for i in np.argwhere(y_train==0).reshape(-1):
    X_resampled.append(X_train[i])
    y_resampled.append(y_train[i])
    sample_index = np.random.randint(0, len(y_one_index)-1)
    X_resampled.append(X_train[y_one_index[sample_index]])
    y_resampled.append(y_train[y_one_index[sample_index]])
X_resampled = np.array(X_resampled)

In [58]:
class GRUCell(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        super(GRUCell, self).__init__()
        def normal(shape):
            return torch.randn(size=shape)*0.0001
        def three():
            return (nn.Parameter(normal((num_inputs, num_hiddens))), 
                    nn.Parameter(normal((num_hiddens, num_hiddens))), 
                    nn.Parameter(torch.zeros(num_hiddens)))
        self.W_xz, self.W_hz, self.b_z = three() # Parameters of update gate
        self.W_xr, self.W_hr, self.b_r = three() # Parameters of reset gate
        self.W_xh, self.W_hh, self.b_h = three() # Parameters of candidate hidden state
        # Parameters of output layer
        self.W_hq = nn.Parameter(normal((num_hiddens, num_outputs)))
        self.b_q = nn.Parameter(torch.zeros(num_outputs))

    def forward(self, X, H):
        Z = torch.sigmoid((X @ self.W_xz) + (H @ self.W_hz) + self.b_z)
        R = torch.sigmoid((X @ self.W_xr) + (H @ self.W_hr) + self.b_r)
        H_tilde = torch.tanh((X @ self.W_xh) + ((R * H) @ self.W_hh) + self.b_h)
        H = Z * H + (1 - Z) * H_tilde
        H.detach()
        Y = torch.sigmoid(H @ self.W_hq + self.b_q)
        return Y, H

In [59]:
class GRUModel(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, imputeMethod, scaleMethod, resampleMethod):
        super(GRUModel, self).__init__()
        self.name = 'GRU'
        self.num_hiddens = num_hiddens
        self.grucell = GRUCell(num_inputs, num_hiddens, num_outputs)
        self.imputeMethod = imputeMethod
        self.scaleMethod = scaleMethod
        self.resampleMethod = resampleMethod

    def forward(self, X, H):
        if H is None:
            H = torch.zeros(X.shape[0], self.num_hiddens)
        X = torch.from_numpy(X.transpose(1, 0, 2)).float()
        for x in X:
            Y, H = self.grucell(x, H)
        return Y, H

In [7]:
def grad_clipping(net, theta):
    params = [p for p in net.parameters() if p.requires_grad]
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [61]:
def train_gru_model(model, X_train, y_train, X_val, y_val, batch_size, lr, num_epoch=25):
    train_loss_all, train_acc_all = [], []
    optimizer = torch.optim.SGD(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()
    threshold = torch.tensor([0.5])
    
    train_num = X_train.shape[0]
    val_num = X_val.shape[0]
    y_train = torch.from_numpy(np.array(y_train)).type(torch.LongTensor)
    y_val = torch.from_numpy(np.array(y_val)).type(torch.LongTensor)
    
    # Compute the AUC of validation set
    output, _ = model(X_val, None)
    print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    for epoch in range(num_epoch):
        print("-" * 40)
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        
        # training stage
        train_loss, train_corrects = 0, 0
        test_loss, test_corrects = 0, 0
        
        for step in range(math.ceil(train_num//batch_size)+1):
            if (step+1)*batch_size <= X_train.shape[0]:
                X = X_train[int(step*batch_size):int((step+1)*batch_size)]
                y = y_train[int(step*batch_size):int((step+1)*batch_size)]
            else:
                X = X_train[int(step*batch_size):]
                y = y_train[int(step*batch_size):]
            
            output, _ = model(X, None)
            y_hat = torch.cat((1-output,output),dim=1)
            loss = criterion(y_hat, y)
            y_predict = (output > threshold).float() * 1
            
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model, 1)
            optimizer.step()
            train_loss += loss.item() * len(y)
            train_corrects += torch.sum(y_predict.squeeze() == y)
        
        # Compute the mean of loss and accuracy for every epoch
        train_loss_all.append(train_loss / train_num)
        train_acc_all.append(train_corrects.double().item() / train_num)
        
        print("Train Loss: {:.4f} Train Acc{:.4f}".format(train_loss_all[-1], train_acc_all[-1]))
        
        if (epoch+1) % 5 == 0:
            # Compute the AUC of validation set
            output, _ = model(X_val, None)
            print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    # save
    torch.save(model.state_dict(), 'E:/WashU/Research/ICU/modelParams/{}_{}_{}_{}_{}_{}_{}_{}.pth'.format(model.imputeMethod, 
                                                                                                          model.scaleMethod, 
                                                                                                          model.resampleMethod, 
                                                                                                          model.name, 
                                                                                                          model.num_hiddens, 
                                                                                                          batch_size, 
                                                                                                          lr, 
                                                                                                          num_epoch))
    
    return model

In [67]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUModel(num_inputs=X_train.shape[2], num_hiddens=512, num_outputs=1, imputeMethod='Mean', scaleMethod='Norm', resampleMethod='RandomBalance').to(device)
model = train_gru_model(model, X_resampled, y_resampled, X_val, y_val, batch_size=32, lr=0.1, num_epoch=5)

Val Auc: 0.5347
----------------------------------------
Epoch 1/5
Train Loss: 0.6792 Train Acc0.5838
----------------------------------------
Epoch 2/5
Train Loss: 0.6740 Train Acc0.5972
----------------------------------------
Epoch 3/5
Train Loss: 0.6720 Train Acc0.5998
----------------------------------------
Epoch 4/5
Train Loss: 0.6711 Train Acc0.6003
----------------------------------------
Epoch 5/5
Train Loss: 0.6697 Train Acc0.6028
Val Auc: 0.6424


In [33]:
f = open('E:/WashU/Research/ICU/Data/train/X_train_sliced_norm_GAN.pkl','rb')
X_train_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/y_train.pkl','rb')
y_train = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_delta_mat.pkl','rb')
train_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/train/train_mask_mat.pkl','rb')
train_mask_mat = pickle.load(f)
f.close()

f = open('E:/WashU/Research/ICU/Data/val/X_val_sliced_norm_GAN.pkl','rb')
X_val_sliced = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/y_val.pkl','rb')
y_val = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_delta_mat.pkl','rb')
val_delta_mat = pickle.load(f)
f.close()
f = open('E:/WashU/Research/ICU/Data/val/val_mask_mat.pkl','rb')
val_mask_mat = pickle.load(f)
f.close()

In [34]:
X_train = np.array(X_train_sliced[:3600])
X_val = np.array(X_train_sliced[3600:])
train_delta = np.array(train_delta_mat[:3600])
val_delta = np.array(train_delta_mat[3600:])
Y_train = np.array(y_train[:3600])
Y_val = np.array(y_train[3600:])
X_resampled = []
y_resampled = []
delta_resampled = []
y_one_index = np.argwhere(Y_train==1).reshape(-1)
for i in np.argwhere(Y_train==0).reshape(-1):
    X_resampled.append(X_train[i])
    y_resampled.append(Y_train[i])
    delta_resampled.append(train_delta[i])
    sample_index = np.random.randint(0, len(y_one_index)-1)
    X_resampled.append(X_train[y_one_index[sample_index]])
    y_resampled.append(Y_train[y_one_index[sample_index]])
    delta_resampled.append(train_delta[y_one_index[sample_index]])
X_resampled = np.array(X_resampled)
delta_resampled = np.array(delta_resampled)

In [5]:
class GRUICell(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs):
        super(GRUICell, self).__init__()
        self.num_inputs = num_inputs
        self.num_hiddens = num_hiddens
        self.num_outputs = num_outputs
        def normal(shape):
            return torch.randn(size=shape)*0.0001
        def three():
            return (nn.Parameter(normal((num_inputs, num_hiddens))), 
                    nn.Parameter(normal((num_hiddens, num_hiddens))), 
                    nn.Parameter(torch.zeros(num_hiddens)))
        self.W_xz, self.W_hz, self.b_z = three() # Parameters of update gate
        self.W_xr, self.W_hr, self.b_r = three() # Parameters of reset gate
        self.W_xh, self.W_hh, self.b_h = three() # Parameters of candidate hidden state
        # Parameters of decay vector
        self.W_beta = nn.Parameter(normal((num_inputs, num_hiddens)))
        self.b_beta = nn.Parameter(torch.zeros(num_hiddens))
        # Parameters of output layer
        self.W_hq = nn.Parameter(normal((num_hiddens, num_outputs)))
        self.b_q = nn.Parameter(torch.zeros(num_outputs))

    def forward(self, X, Delta, H):
        beta = torch.exp(torch.minimum(torch.zeros(self.num_hiddens), Delta @ self.W_beta + self.b_beta))
        H = beta * H
        Z = torch.sigmoid((X @ self.W_xz) + (H @ self.W_hz) + self.b_z)
        R = torch.sigmoid((X @ self.W_xr) + (H @ self.W_hr) + self.b_r)
        H_tilde = torch.tanh((X @ self.W_xh) + ((R * H) @ self.W_hh) + self.b_h)
        H = Z * H + (1 - Z) * H_tilde
        H.detach()
        Y = torch.sigmoid(H @ self.W_hq + self.b_q)
        return Y, H

In [21]:
class GRUIModel(nn.Module):
    def __init__(self, num_inputs, num_hiddens, num_outputs, imputeMethod, scaleMethod, resampleMethod):
        super(GRUIModel, self).__init__()
        self.name = 'GRUI'
        self.num_hiddens = num_hiddens
        self.gruicell = GRUICell(num_inputs, num_hiddens, num_outputs)
        self.imputeMethod = imputeMethod
        self.scaleMethod = scaleMethod
        self.resampleMethod = resampleMethod

    def forward(self, X, Delta, H):
        if H is None:
            H = torch.zeros(X.shape[1], self.num_hiddens)
        for index in range(X.shape[0]):
            Y, H = self.gruicell(X[index], Delta[index], H)
        return Y, H

In [134]:
def train_grui_model(model, X_train, y_train, train_delta_mat, X_val, y_val, val_delta_mat, batch_size, lr, num_epoch=25):
    train_loss_all, train_acc_all = [], []
    optimizer = torch.optim.SGD(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()
    threshold = torch.tensor([0.5])
    
    train_num = X_train.shape[0]
    val_num = X_val.shape[0]
    X_train = torch.from_numpy(X_train.transpose(1, 0, 2)).float()
    X_val = torch.from_numpy(X_val.transpose(1, 0, 2)).float()
    train_delta_mat = torch.from_numpy(train_delta_mat.transpose(1, 0, 2)).float()
    val_delta_mat = torch.from_numpy(val_delta_mat.transpose(1, 0, 2)).float()
    y_train = torch.from_numpy(np.array(y_train)).type(torch.LongTensor)
    y_val = torch.from_numpy(np.array(y_val)).type(torch.LongTensor)
    
    # Compute the AUC of validation set
    output, _ = model(X_val, val_delta_mat, None)
    print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    for epoch in range(num_epoch):
        print("-" * 40)
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        
        # training stage
        train_loss, train_corrects = 0, 0
        test_loss, test_corrects = 0, 0
        
        for step in range(train_num//batch_size+1):
            if (step+1)*batch_size <= train_num:
                X = X_train[:, int(step*batch_size):int((step+1)*batch_size), :]
                y = y_train[int(step*batch_size):int((step+1)*batch_size)]
                Delta = train_delta_mat[:, int(step*batch_size):int((step+1)*batch_size), :]
            else:
                X = X_train[:, int(step*batch_size):, :]
                y = y_train[int(step*batch_size):]
                Delta = train_delta_mat[:, int(step*batch_size):, :]
            
            output, _ = model(X, Delta, None)
            y_hat = torch.cat((1-output,output),dim=1)
            loss = criterion(y_hat, y)
            y_predict = (output > threshold).float() * 1
            
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model, 1)
            optimizer.step()
            train_loss += loss.item() * len(y)
            train_corrects += torch.sum(y_predict.squeeze() == y)
        
        # Compute the mean of loss and accuracy for every epoch
        train_loss_all.append(train_loss / train_num)
        train_acc_all.append(train_corrects.double().item() / train_num)
        
        print("Train Loss: {:.4f} Train Acc{:.4f}".format(train_loss_all[-1], train_acc_all[-1]))
        
        if (epoch+1) % 5 == 0:
            # Compute the AUC of validation set
            output, _ = model(X_val, val_delta_mat, None)
            print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    # save
    torch.save(model.state_dict(), 'E:/WashU/Research/ICU/modelParams/{}_{}_{}_{}_{}_{}_{}_{}.pth'.format(model.imputeMethod, 
                                                                                                          model.scaleMethod, 
                                                                                                          model.resampleMethod, 
                                                                                                          model.name, 
                                                                                                          model.num_hiddens, 
                                                                                                          batch_size, 
                                                                                                          lr, 
                                                                                                          num_epoch))
    
    return model

In [32]:
y_val

array([], dtype=int32)

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUIModel(num_inputs=X_train.shape[2], num_hiddens=64, num_outputs=1, imputeMethod='GAN', scaleMethod='Norm', resampleMethod='RandomBalance').to(device)
model = train_grui_model(model, X_resampled, y_resampled, delta_resampled, X_val, Y_val, val_delta, batch_size=32, lr=0.1, num_epoch=50)

Val Auc: 0.5097
----------------------------------------
Epoch 1/50
Train Loss: 0.6931 Train Acc0.6130
----------------------------------------
Epoch 2/50
Train Loss: 0.6931 Train Acc0.6229
----------------------------------------
Epoch 3/50
Train Loss: 0.6931 Train Acc0.5706
----------------------------------------
Epoch 4/50
Train Loss: 0.6930 Train Acc0.5229
----------------------------------------
Epoch 5/50
Train Loss: 0.6928 Train Acc0.5937
Val Auc: 0.7122
----------------------------------------
Epoch 6/50
Train Loss: 0.6922 Train Acc0.6202
----------------------------------------
Epoch 7/50
Train Loss: 0.6901 Train Acc0.6255
----------------------------------------
Epoch 8/50
Train Loss: 0.6830 Train Acc0.6391
----------------------------------------
Epoch 9/50
Train Loss: 0.6670 Train Acc0.6473
----------------------------------------
Epoch 10/50
Train Loss: 0.6480 Train Acc0.6508
Val Auc: 0.7326
----------------------------------------
Epoch 11/50
Train Loss: 0.6333 Train Acc

KeyboardInterrupt: 

In [37]:
output, _ = model(X_val, None)
print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
output, _ = model(X_test, None)
print('test Auc: {:.4f}'.format(metrics.roc_auc_score(y_test, output.detach().numpy())))

Val Auc: 0.7927


ValueError: axes don't match array

In [12]:
y_predict, _ = model(X_val, val_delta_mat, None)
threshold = torch.tensor([0.5])
y_predict = (y_predict > threshold).float() * 1
# score1
TP = 0
FN = 0
FP = 0
TN = 0
for i in range (len(y_val)):
    if (y_val[i] == 1) & (y_predict.detach().numpy()[i] == 1):
        TP = TP + 1
    if (y_val[i] == 1) & (y_predict.detach().numpy()[i] == 0):
        FN = FN + 1
    if (y_val[i] == 0) & (y_predict.detach().numpy()[i] == 1):
        FP = FP + 1
    if (y_val[i] == 0) & (y_predict.detach().numpy()[i] == 0):
        TN = TN + 1
print(min(TP/(TP+FN),TP/(TP+FP)))

0.3706972639011474


In [1]:
y_predict, _ = model(X_val, val_delta_mat, None)
a = y_predict.detach().numpy()
b = np.reshape(np.array(y_val),(np.array(y_val).size,1))
a[a > 0.99] = 0.99
a[a < 0.01] = 0.01
c = np.append(a, b, axis=1)
c =c[np.argsort(c[:,0])]
score2 = 0
for i in range (9):
    d = c[400*i:(400*i+399),:]
    pi_g = np.mean(d[:,0])
    N_g = 400
    O_g = np.count_nonzero(d[:,1])
    e = d[:,0]
    e[e<0.5]=0
    E_g = np.count_nonzero(e)
    score2 = score2 + (O_g - E_g)*(O_g - E_g)/(N_g * pi_g * (1-pi_g) + 0.001)
d = c[3600:3992,:]
pi_g = np.mean(d[:,0])
N_g = 393
O_g = np.count_nonzero(d[:,1])
e = d[:,0]
e[e<0.5]=0
E_g = np.count_nonzero(e)
score2 = score2 + (O_g - E_g)*(O_g - E_g)/(N_g * pi_g * (1-pi_g) + 0.001)
score2

NameError: name 'model' is not defined

In [49]:
import os
import random
class ReadImputedPhysionetData:
    def __init__(self, dataPath ):
        #一个文件一个batch，但需要注意，x,y,delta之间的匹配
        #例子： batch1y,batch1x,batch1delta
        #batchid从1开始
        self.files = os.listdir(dataPath)
        self.dataPath=dataPath
        self.count=int(len(self.files)/3)
        
    def load(self):
        count=int(self.count)
        self.x=[]
        self.y=[]
        self.delta=[]
        self.x_lengths=[]
        self.m=[]
        for i in range(1,count+1):
            file_x=open(os.path.join(self.dataPath,"batch"+str(i)+"x"))
            file_y=open(os.path.join(self.dataPath,"batch"+str(i)+"y"))
            file_delta=open(os.path.join(self.dataPath,"batch"+str(i)+"delta"))
            this_x,this_lengths=self.readx(file_x)
            self.x.extend(this_x)
            self.x_lengths.extend(this_lengths)
            self.y.extend(self.ready(file_y))
            this_delta,this_m=self.readdelta(file_delta)
            self.delta.extend(this_delta)
            self.m.extend(this_m)
            file_x.close()
            file_y.close()
            file_delta.close()
        self.maxLength=len(self.x[0])
        
        
    def readx(self,x):
        this_x=[]
        this_lengths=[]
        count=1
        for line in x.readlines():
            if count==1:
                words=line.strip().split(",")
                for w in words:
                    if w=='':
                        continue
                    this_lengths.append(int(w))
            else:
                if "end" in line:
                    continue
                if "begin" in line:
                    d=[]
                    this_x.append(d)
                else:
                    words=line.strip().split(",")
                    if words[0]=='':
                        continue
                    oneclass=[]
                    for w in words:
                        if w=='':
                            continue
                        oneclass.append(float(w))
                    this_x[-1].append(oneclass)
            count+=1
        return this_x,this_lengths
    
    def ready(self,y):
        this_y=[]
        for line in y.readlines():
            d=[]
            words=line.strip().split(",")
            for w in words:
                if w=='':
                    continue
                d.append(int(w))
            this_y.append(d)
        return this_y
    
    def readdelta(self,delta):
        this_delta=[]
        this_m=[]
        for line in delta.readlines():
            if "end" in line:
                continue
            if "begin" in line:
                d=[]
                this_delta.append(d)
                t=[]
                this_m.append(t)
            else:
                words=line.strip().split(",")
                oneclass=[]
                onem=[]
                for i in range(len(words)):
                    w=words[i]
                    if w=='':
                        continue
                    oneclass.append(float(w))
                    if i==0 or float(w) >0:
                        onem.append(1.0)
                    else:
                        onem.append(0.0)
                this_delta[-1].append(oneclass)
                this_m[-1].append(onem)
        return this_delta,this_m
    
    def shuffle(self,batchSize=128,isShuffle=False):
        self.batchSize=batchSize
        if isShuffle:
            c = list(zip(self.x,self.y,self.m,self.delta,self.x_lengths))
            random.shuffle(c)
            self.x,self.y,self.m,self.delta,self.x_lengths=zip(*c)
            
    def nextBatch(self):
        i=1
        while i*self.batchSize<=len(self.x):
            x=[]
            y=[]
            m=[]
            delta=[]
            x_lengths=[]
            for j in range((i-1)*self.batchSize,i*self.batchSize):
                x.append(self.x[j])
                y.append(self.y[j])
                m.append(self.m[j])
                delta.append(self.delta[j])
                x_lengths.append(self.x_lengths[j])
            i+=1
            yield  x,y,[0.0]*len(self.x[0][0]),m,delta,x_lengths,x,0,0,0

dt=ReadImputedPhysionetData("E:/WashU/Research/Multivariate-Time-Series-Imputation-with-Generative-Adversarial-Networks/Gan_Imputation/imputation_train_results/WGAN_no_mask/30_8_128_64_0.001_400_True_True_True_0.15_0.5")
dt.load()
dt_test=ReadImputedPhysionetData("E:/WashU/Research/Multivariate-Time-Series-Imputation-with-Generative-Adversarial-Networks/Gan_Imputation/imputation_test_results/WGAN_no_mask/30_8_128_64_0.001_400_True_True_True_0.15_0.5")
dt_test.load()

In [223]:
X_train = np.array(dt.x)
X_train.reshape((X_train.shape[0],X_train.shape[1],41))
y_train = np.array(dt.y[::2])[:,1]
delta_train = []
for i in range(len(dt.delta)):
    delta_train.append(np.array(dt.delta[0][1:-1:2]))
delta_train = np.array(delta_train)
X_test = np.array(dt_test.x)
X_test.reshape((X_test.shape[0],X_test.shape[1],41))
y_test = np.array(dt_test.y[::2])[:,1]
delta_test = []
for i in range(len(dt_test.delta)):
    delta_test.append(np.array(dt_test.delta[0][1:-1:2]))
delta_test = np.array(delta_test)
X_resampled = []
y_resampled = []
delta_resampled = []
y_one_index = np.argwhere(y_train==1).reshape(-1)
for i in np.argwhere(y_train==0).reshape(-1):
    X_resampled.append(X_train[i])
    y_resampled.append(y_train[i])
    delta_resampled.append(delta_train[i])
    sample_index = np.random.randint(0, len(y_one_index)-1)
    X_resampled.append(X_train[y_one_index[sample_index]])
    y_resampled.append(y_train[y_one_index[sample_index]])
    delta_resampled.append(delta_train[y_one_index[sample_index]])
X_resampled = np.array(X_resampled)
delta_resampled = np.array(delta_resampled)

In [220]:
def train_grui_model(model, X_train, y_train, train_delta_mat, X_val, y_val, val_delta_mat, batch_size, lr, num_epoch=25):
    train_loss_all, train_acc_all = [], []
    optimizer = torch.optim.SGD(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()
    threshold = torch.tensor([0.5])
    
    train_num = X_train.shape[0]
    val_num = X_val.shape[0]
    X_train = torch.from_numpy(X_train.transpose(1, 0, 2)).float()
    X_val = torch.from_numpy(X_val.transpose(1, 0, 2)).float()
    train_delta_mat = torch.from_numpy(train_delta_mat.transpose(1, 0, 2)).float()
    val_delta_mat = torch.from_numpy(val_delta_mat.transpose(1, 0, 2)).float()
    y_train = torch.from_numpy(np.array(y_train)).type(torch.LongTensor)
    y_val = torch.from_numpy(np.array(y_val)).type(torch.LongTensor)
    
    # Compute the AUC of validation set
    output, _ = model(X_val, val_delta_mat, None)
    print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    for epoch in range(num_epoch):
        print("-" * 40)
        print('Epoch {}/{}'.format(epoch+1, num_epoch))
        
        # training stage
        train_loss, train_corrects = 0, 0
        test_loss, test_corrects = 0, 0
        
        for step in range(math.ceil(train_num/batch_size)):
            if (step+1)*batch_size <= train_num:
                X = X_train[:, int(step*batch_size):int((step+1)*batch_size), :]
                y = y_train[int(step*batch_size):int((step+1)*batch_size)]
                Delta = train_delta_mat[:, int(step*batch_size):int((step+1)*batch_size), :]
            else:
                X = X_train[:, int(step*batch_size):, :]
                y = y_train[int(step*batch_size):]
                Delta = train_delta_mat[:, int(step*batch_size):, :]
            
            output, _ = model(X, Delta, None)
            y_hat = torch.cat((1-output,output),dim=1)
            loss = criterion(y_hat, y)
            y_predict = (output > threshold).float() * 1
            
            optimizer.zero_grad()
            loss.backward()
            grad_clipping(model, 1)
            optimizer.step()
            train_loss += loss.item() * len(y)
            train_corrects += torch.sum(y_predict.squeeze() == y)
        
        # Compute the mean of loss and accuracy for every epoch
        train_loss_all.append(train_loss / train_num)
        train_acc_all.append(train_corrects.double().item() / train_num)
        
        print("Train Loss: {:.4f} Train Acc{:.4f}".format(train_loss_all[-1], train_acc_all[-1]))
        
        if (epoch+1) % 5 == 0:
            # Compute the AUC of validation set
            output, _ = model(X_val, val_delta_mat, None)
            print('Val Auc: {:.4f}'.format(metrics.roc_auc_score(y_val, output.detach().numpy())))
    
    # save
    torch.save(model.state_dict(), 'E:/WashU/Research/ICU/modelParams/{}_{}_{}_{}_{}_{}_{}_{}.pth'.format(model.imputeMethod, 
                                                                                                          model.scaleMethod, 
                                                                                                          model.resampleMethod, 
                                                                                                          model.name, 
                                                                                                          model.num_hiddens, 
                                                                                                          batch_size, 
                                                                                                          lr, 
                                                                                                          num_epoch))
    
    return model

In [210]:
outputs, _ = model(torch.from_numpy(X_train.transpose(1, 0, 2)).float(), torch.from_numpy(delta_train.transpose(1, 0, 2)).float(), None)
y_hat = torch.cat((1-outputs,outputs),dim=1)
criterion = nn.CrossEntropyLoss()
loss = criterion(y_hat, torch.from_numpy(np.array(y_train)).type(torch.LongTensor))

In [217]:
X_train.shape

(3584, 48, 41)

In [225]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GRUIModel(num_inputs=X_train.shape[2], num_hiddens=64, num_outputs=1, imputeMethod='GAN', scaleMethod='GAN', resampleMethod='None').to(device)
model = train_grui_model(model, X_resampled, y_resampled, delta_resampled, X_test, y_test, delta_test, batch_size=32, lr=0.1, num_epoch=200)

Val Auc: 0.4983
----------------------------------------
Epoch 1/200
Train Loss: 0.6931 Train Acc0.5826
----------------------------------------
Epoch 2/200
Train Loss: 0.6931 Train Acc0.6037
----------------------------------------
Epoch 3/200
Train Loss: 0.6931 Train Acc0.6106
----------------------------------------
Epoch 4/200
Train Loss: 0.6931 Train Acc0.6111
----------------------------------------
Epoch 5/200
Train Loss: 0.6929 Train Acc0.6118
Val Auc: 0.6713
----------------------------------------
Epoch 6/200
Train Loss: 0.6908 Train Acc0.6121
----------------------------------------
Epoch 7/200
Train Loss: 0.6788 Train Acc0.6212
----------------------------------------
Epoch 8/200
Train Loss: 0.6472 Train Acc0.6609
----------------------------------------
Epoch 9/200
Train Loss: 0.6081 Train Acc0.7068
----------------------------------------
Epoch 10/200
Train Loss: 0.5788 Train Acc0.7247
Val Auc: 0.7855
----------------------------------------
Epoch 11/200
Train Loss: 0.568

Train Loss: 0.4801 Train Acc0.8324
Val Auc: 0.8067
----------------------------------------
Epoch 91/200
Train Loss: 0.4804 Train Acc0.8327
----------------------------------------
Epoch 92/200
Train Loss: 0.4809 Train Acc0.8334
----------------------------------------
Epoch 93/200
Train Loss: 0.4784 Train Acc0.8344
----------------------------------------
Epoch 94/200
Train Loss: 0.4781 Train Acc0.8358
----------------------------------------
Epoch 95/200
Train Loss: 0.4760 Train Acc0.8376
Val Auc: 0.8063
----------------------------------------
Epoch 96/200
Train Loss: 0.4760 Train Acc0.8373
----------------------------------------
Epoch 97/200
Train Loss: 0.4751 Train Acc0.8386
----------------------------------------
Epoch 98/200
Train Loss: 0.4750 Train Acc0.8391
----------------------------------------
Epoch 99/200
Train Loss: 0.4737 Train Acc0.8407
----------------------------------------
Epoch 100/200
Train Loss: 0.4712 Train Acc0.8423
Val Auc: 0.7957
--------------------------

Train Loss: 0.4491 Train Acc0.8649
----------------------------------------
Epoch 179/200
Train Loss: 0.4497 Train Acc0.8644
----------------------------------------
Epoch 180/200
Train Loss: 0.4535 Train Acc0.8603
Val Auc: 0.7957
----------------------------------------
Epoch 181/200
Train Loss: 0.4461 Train Acc0.8681
----------------------------------------
Epoch 182/200
Train Loss: 0.4465 Train Acc0.8676
----------------------------------------
Epoch 183/200
Train Loss: 0.4474 Train Acc0.8663
----------------------------------------
Epoch 184/200
Train Loss: 0.4480 Train Acc0.8663
----------------------------------------
Epoch 185/200
Train Loss: 0.4479 Train Acc0.8662
Val Auc: 0.7895
----------------------------------------
Epoch 186/200
Train Loss: 0.4471 Train Acc0.8665
----------------------------------------
Epoch 187/200
Train Loss: 0.4437 Train Acc0.8704
----------------------------------------
Epoch 188/200
Train Loss: 0.4456 Train Acc0.8683
---------------------------------

In [226]:
def Score1(method_Pred, ytest):
    score1 = 0
    TP = 0
    FN = 0
    FP = 0
    TN = 0
    for i in range (len(ytest)):
        if (ytest[i] == 1) & (method_Pred[i] == 1):
            TP = TP + 1
        if (ytest[i] == 1) & (method_Pred[i] == 0):
            FN = FN + 1
        if (ytest[i] == 0) & (method_Pred[i] == 1):
            FP = FP + 1
        if (ytest[i] == 0) & (method_Pred[i] == 0):
            TN = TN + 1
    if ((TP == 0) & (FN == 0)):
        Se = 0
    else:
        Se = TP/(TP+FN)
        
    if ((TP == 0) & (FP == 0)):
        P = 0
    else:
        P = TP/(TP+FP)
    
    if Se > P:
        score1 = P
    else:
        score1 = Se
    return score1

In [233]:
output, _ = model(X_test, delta_test, None)
y_predict = (output > 0.5).float() * 1
Score1(y_predict, y_test)

0.2804878048780488